<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Single-cell-tracks" data-toc-modified-id="Single-cell-tracks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Single cell tracks</a></span><ul class="toc-item"><li><span><a href="#Analyse-multiple-data-sets" data-toc-modified-id="Analyse-multiple-data-sets-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Analyse multiple data sets</a></span><ul class="toc-item"><li><span><a href="#Process-data" data-toc-modified-id="Process-data-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Process data</a></span></li></ul></li></ul></li><li><span><a href="#Old-code-(ignore-below)" data-toc-modified-id="Old-code-(ignore-below)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Old code (ignore below)</a></span><ul class="toc-item"><li><span><a href="#Spatial-correlations" data-toc-modified-id="Spatial-correlations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Spatial correlations</a></span><ul class="toc-item"><li><span><a href="#Spatial-velocity-correlations" data-toc-modified-id="Spatial-velocity-correlations-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Spatial velocity correlations</a></span></li><li><span><a href="#Check-that-the-number-of-pairs-of-cells-in-each-of-the-bins-is-roughly-equal" data-toc-modified-id="Check-that-the-number-of-pairs-of-cells-in-each-of-the-bins-is-roughly-equal-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Check that the number of pairs of cells in each of the bins is roughly equal</a></span></li><li><span><a href="#Pairwise-distance-statistics" data-toc-modified-id="Pairwise-distance-statistics-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Pairwise distance statistics</a></span></li></ul></li></ul></li></ul></div>

# Single cell tracks

## Analyse multiple data sets

In [ ]:
# --- Input parameters ---
video_labels = ["BAPN_E14.0_211223", "BAPN_E14.0_221114"];
pix_per_micron_all = [3.368, 3.376]; # Number of pixels per micron
names_compare = ["Bone Front", "In Bone", "Further In Bone"];

video_image_sizes = np.array([[1136.28, 1133.31], [1158.77, 1128.85]]); # in microns

delta_t = 1/6; # time interval between slices in hrs


In [ ]:
data_paths = ["/Users/dang/Documents/Projects/Tabler_skull/Data_Live_Imaging/211223 BAPN E14.0 live imaging/cell_tracks/",
              "/Users/dang/Documents/Projects/Tabler_skull/Data_Live_Imaging/221114 BAPN E14.0 live imaging/cell_tracks/"]
fnames = ["1_interface_points.csv",
         "2_bulk_moving_cells_points.csv"]

N_videos = len(data_paths)
N_files_per_video = len(fnames)

loaded_data_all = [[None]*len(fnames) for _ in range(len(data_paths))];

for i in range(len(data_paths)):
    for j in range(len(fnames)):
        loaded_data_all[i][j] = pd.read_csv(os.path.join(data_paths[i], fnames[j]))


In [ ]:
# Load files
# data_path = "/Users/dang/Documents/Projects/Tabler_skull/Data_Adrian_cell_tracks/"
# fname_templates = ["Results from MaxProjected_Stitched_Blurred_190305 ML1 OsxGFP Explant Video 1 (0-12hr) in um per min.csv",
#                   "Results from MaxProjected_Stitched_Blurred_190314 ML1 OsxGFP Explant Video 1 (0-8hr) in um per sec.csv",
#                    "Results from MaxProjected_Stitched_Blurred_200120 Live Imaging OsxGFP E1375 12 hours in um per sec.csv",
#                   "Results from MAX_BLUR_20200414AQR_osx_E13 in um per min.csv"]
# filenames_all = [["{} {}".format(x,y) for x in names_compare] for y in fname_templates ];
# N_videos = len(fname_templates);
# N_files_per_video = len(names_compare);
# video_image_sizes_pix = [[int(pix_per_micron_all[i]*video_image_sizes[i, j]) for j in range(2)] for i in range(N_videos)]; # in pixels

# loaded_data_all = [[None]*N_files_per_video for _ in range(N_videos)];
# for i in range(N_videos):    
#     subfolder = "Video {}/".format(i+1);
#     for j in range(N_files_per_video):
#         loaded_data_all[i][j] = pd.read_csv(data_path+subfolder+filenames_all[i][j]);

In [ ]:
loaded_data_all[1][1]

### Process data

In [ ]:
# Total number of cells
N_cells_all = [ [len(np.unique(loaded_data_all[i][j].TID)) for j in range(N_files_per_video) ] for i in range(N_videos)];
print("Number of cells = " + str(N_cells_all))

# Number of slices per cell
N_slices_all = [ [len(np.unique(loaded_data_all[i][j]["t [min]"])) for j in range(N_files_per_video)] for i in range(N_videos)];
print("Max. number of slices(=frames) per cell = " + str(N_slices_all))

# Old code (ignore below)

## Spatial correlations
How do correlations between cell motions depend on the spatial locations and distances between cells? <br>

### Spatial velocity correlations
Define $C_{VV}(x_i, x_j) = \langle v(x_i, t)\, v(x_j, t) \rangle$ as the spatial correlation function of cell velocities. Assume $C_{VV}(x_i, x_j) = C_{VV}(x_i-x_j)$ and average over cells with the same distance (in practice, bin together pairs of cells that have similar distance to each other). 

In [ ]:
nbins = 15;
edges = np.linspace(0, 2000, nbins+1)
#t_selected = np.arange(0, N_slices_all[i1][i2]-1, 6) # <--- adjust plotted times
t_selected = np.array([0, 1, 2, 3, 4, 5, 6])*6; #np.arange(0, N_slices_all[i1][i2]-1, 6) 
normalize = 1;
ylabel_extra='';
if normalize:
    ylabel_extra = "direction"
    
corr_v_all_all = np.zeros([N_videos, N_files_per_video, len(t_selected), nbins]);

fig, axs = plt.subplots(N_videos, N_files_per_video, figsize=(12, 12), dpi=100)
for i1 in range(N_videos):
    #[ix, iy] = divmod(i1, 2); # positions of plots
    #pix_per_micron = pix_per_micron_all[i1];
    for i2 in range(N_files_per_video):
        t_idx=0;
        for t in t_selected:
            X_all = X_all_all[i1][i2][:, t];
            Y_all = Y_all_all[i1][i2][:, t];

            # filter out cells which are not tracked
            if t<N_slices_all[i1][i2]:
                cell_filter = ~np.isnan( X_all_all[i1][i2][:, t+1] ); # cell posiiton still defined the next time step
            else:
                cell_filter = ~np.isnan( X_all_all[i1][i2][:, t] );
            X_all = X_all[cell_filter];
            Y_all = Y_all[cell_filter];

            # pairwise distances between cells
            incl_self = 0; # include self-pairs of the form [i,i]?

            # calculate pairwise distances
            X_all_paired = np.array([ [X_all[i], X_all[j]] for i in range(len(X_all)) for j in range(i+incl_self)]); # loop j over range(i+1) to include self-pairs, or range(i) to exclude self-pairs
            Y_all_paired = np.array([ [Y_all[i], Y_all[j]] for i in range(len(Y_all)) for j in range(i+incl_self)]);
            def f(x, y):
                return np.sqrt((x[0]-x[1])**2 + (y[0]-y[1])**2)
            pairwise_dist_all = np.array(list(map(f, X_all_paired, Y_all_paired )) );

            # retrieve velocities
            this_velocities_all = velocities_all_all[i1][i2][cell_filter, t, :];

            # linear to pair indices
            pair_idx = np.array([ [i, j] for i in range(len(X_all)) for j in range(i+incl_self)]); 

            # define bin edges, bin weights
            # everything that falls outside of the edges of the pre-defined bins is neglected
            hist_data, edges = np.histogram(pairwise_dist_all, bins = edges);
            compare_bin_edges = np.array([pairwise_dist_all[i] <= edges for i in range(len(pairwise_dist_all))]);
            bin_idx = np.array(list(map( np.argmax, compare_bin_edges))); # indices of the bins of all the data

            # calculate correlations
            corr_v_all = np.zeros(len(edges)-1); # correlation function
            temp_sum = [];
            for i in range(1, len(edges)):
                bin_i_idx = np.argwhere(bin_idx==i).flatten(); # linear indices that fall in bin i
                temp_sum.append(len(bin_i_idx));
                bin_i_pair_idx = pair_idx[bin_i_idx]; # pair indices that fall in bin i
                this_corr_v = np.zeros( len(bin_i_pair_idx) );
                for j in range(len(bin_i_pair_idx)):
                    cell_1=bin_i_pair_idx[j][0]
                    cell_2=bin_i_pair_idx[j][1]
                    v1 = this_velocities_all[cell_1];
                    v2 = this_velocities_all[cell_2];
                    #corr_v_all[i] += np.inner(v1, v2)/np.linalg.norm(v1)/np.linalg.norm(v2); 
                    corr_temp = np.inner(v1, v2);
                    if normalize and np.linalg.norm(v1)!=0 and np.linalg.norm(v2)!=0:
                        corr_temp = corr_temp/np.linalg.norm(v1)/np.linalg.norm(v2);  
                    this_corr_v[j] = corr_temp;
                #corr_v_all[i] = corr_v_all[i]/len(bin_i_pair_idx);
                corr_v_all[i-1] = np.nanmean(this_corr_v);

            # plot results
            bincenters = (edges[1:]+edges[:-1])/2;
            #axs[i1, i2].plot(bincenters, corr_v_all, alpha=0.5);
            axs[i1, i2].plot(bincenters/pix_per_micron, corr_v_all, color=np.ones(3)*0.8*(t/t_selected[-1]), alpha=0.5)
            
            # store all results
            corr_v_all_all[i1, i2, t_idx, :] = corr_v_all;
            t_idx += 1;
            
        this_title = 'Video {}, {}'.format(i1+1, names_compare[i2]);
        axs[i1, i2].set(title=this_title, xlabel = 'Distance ($\mu m$)', ylabel = 'Velocity {}\n correlation'.format(ylabel_extra))
        if normalize:
            axs[i1, i2].set(ylim = (-1, 1))
plt.subplots_adjust(hspace=0.5, wspace=0.4)
plt.show()
print( "Lighter colors = later times" )

# Display legend
# axs[1].plot(np.nan, label='t={}'.format(t/6));
# axs[1].legend(loc='upper left')
# axs[1].set(xticks=[], yticks=[]);
# axs[1].axis('off')
# plt.xlabel('Distance ($\mu m$)')
# plt.ylabel('Velocity correlation')
# plt.ylim([-1, 1])

In [ ]:
t_idx_selected = np.arange(len(t_selected)-1); #np.array([0, 2, 4, 6])*6; #np.arange(0, N_slices_all[i1][i2]-1, 6) 

fig, axs = plt.subplots(N_videos, len(t_idx_selected)+1, figsize=(4*len(t_idx_selected), 3*N_videos), dpi=100)

for i1 in range(N_videos):
    #[ix, iy] = divmod(i1, 2); # positions of plots
    #pix_per_micron = pix_per_micron_all[i1];
    for t_idx in range(len(t_idx_selected)):
        t = t_selected[t_idx];
        for i2 in range(N_files_per_video):
            corr_v_all = corr_v_all_all[i1, i2, t_idx, :];
            
            # plot results
            bincenters = (edges[1:]+edges[:-1])/2;
            #axs[i1, i2].plot(bincenters, corr_v_all, alpha=0.5);
            axs[i1, t_idx].plot(bincenters/pix_per_micron, corr_v_all, color=def_colors[i2], alpha=0.5)
            
        this_title = 'Video {}, t={} hr'.format(i1+1, t/6);
        axs[i1, t_idx].set(title=this_title, xlabel = 'Distance ($\mu m$)', ylabel = 'Velocity {}\n correlation'.format(ylabel_extra))
        if normalize:
            axs[i1, t_idx].set_ylim( (-1, 1) )

# ---- Legend ----
for row in range(N_videos):
    for i, this_label in enumerate(names_compare):
        axs[row, t_idx+1].plot(np.nan, label=this_label, color=def_colors[i])
    axs[row, t_idx+1].legend(loc='upper left')
    axs[row, t_idx+1].axis('off')

plt.subplots_adjust(hspace=0.5, wspace=0.4)
plt.tight_layout()

plt.show()

<b>Conclusion</b>: no significant spatial dependence of velocity correlations. Generally, cells far away from each other are just as likely to move in the same direction as cells close to each other.

In [ ]:
t_idx_selected = np.arange(len(t_selected)-1); #np.array([0, 2, 4, 6])*6; #np.arange(0, N_slices_all[i1][i2]-1, 6) 
video_sel = [0, 1, 2, 3];

fig, axs = plt.subplots(1, len(t_idx_selected)+1, figsize=(16, 3), dpi=100)

for t_idx in range(len(t_idx_selected)):
    t = t_selected[t_idx];
    for i2 in range(N_files_per_video):
        corr_v_all_avg = np.mean( corr_v_all_all[:, i2, t_idx, :], axis=0 )
        corr_v_all_sem = np.std( corr_v_all_all[:, i2, t_idx, :], axis=0 )/np.sqrt(N_videos);
        
        # plot results
        bincenters = (edges[1:]+edges[:-1])/2;
        #axs[i1, i2].plot(bincenters, corr_v_all, alpha=0.5);
        axs[t_idx].plot(bincenters/pix_per_micron, corr_v_all_avg, color=colors_cats[i2], alpha=0.5)
        axs[t_idx].fill_between(bincenters/pix_per_micron, corr_v_all_avg-corr_v_all_sem, corr_v_all_avg+corr_v_all_sem, color=colors_cats[i2], alpha=0.2)

    this_title = 't={} hr'.format(t/6);
    axs[t_idx].set(title=this_title, xlabel = 'Distance ($\mu m$)', ylabel = '{}\n correlation'.format(ylabel_extra));
    if normalize:
        axs[t_idx].set_ylim( (-0.2, 1) )

# ---- Legend ----
for i, this_label in enumerate(names_compare):
    axs[len(t_idx_selected)].plot(np.nan, label=this_label, color=colors_cats[i])
axs[len(t_idx_selected)].legend(loc='upper left')
axs[len(t_idx_selected)].axis('off')
        
plt.subplots_adjust(hspace=0.5, wspace=0.4)
plt.tight_layout()

plt.show()

### Check that the number of pairs of cells in each of the bins is roughly equal
(or at least, that there are no bins which have very few cells)

In [ ]:
t = 0;
#------------ Recompute bins ------------------ 
X_all = X_all_all[i1][i2][:, t]; Y_all = Y_all_all[i1][i2][:, t];
if t<N_slices_all[i1][i2]:
    cell_filter = ~np.isnan( X_all_all[i1][i2][:, t+1] ); # cell posiiton still defined the next time step
else:
    cell_filter = ~np.isnan( X_all_all[i1][i2][:, t] );
X_all = X_all[cell_filter]; Y_all = Y_all[cell_filter];
X_all_paired = np.array([ [X_all[i], X_all[j]] for i in range(len(X_all)) for j in range(i+incl_self)]); # loop j over range(i+1) to include self-pairs, or range(i) to exclude self-pairs
Y_all_paired = np.array([ [Y_all[i], Y_all[j]] for i in range(len(Y_all)) for j in range(i+incl_self)]);
def f(x, y):
    return np.sqrt((x[0]-x[1])**2 + (y[0]-y[1])**2)
pairwise_dist_all = np.array(list(map(f, X_all_paired, Y_all_paired )) );
this_velocities_all = velocities_all_all[i1][i2][cell_filter, t, :];
pair_idx = np.array([ [i, j] for i in range(len(X_all)) for j in range(i+incl_self)]); 
hist_data, edges = np.histogram(pairwise_dist_all, bins = edges);
#----------------------------------------------

plt.figure(dpi=100);
bincenters = (edges[1:]+edges[:-1])/2
plt.bar(range(len(hist_data)), hist_data)
#plt.plot([0, 900], 20*np.ones(2), 'r--')
plt.xlabel('Bin')
plt.ylabel('Count')
plt.show()

In [ ]:
# --- Troubleshoot ---
print( hist_data )
print( temp_sum )
print(np.sum(bin_idx==0) )
#print( np.argwhere( np.array(list(map( np.argmax, compare_bin_edges))) == 0 ) )
this_bin = 2;
print( pairwise_dist_all[this_bin] )
print( compare_bin_edges[this_bin] )
print( pair_idx[this_bin] )
# print( X_all_all[i1][i2][10, t], X_all_all[i1][i2][5, t] )
# print( Y_all_all[i1][i2][10, t], Y_all_all[i1][i2][5, t] )

### Pairwise distance statistics

Before plotting the correlation functions, we should examine the distribution of all pairwise distances of the tracked cells. <br>

In [ ]:
# --- Plot for one snapshot ---
i1=0;
i2=1;
t=2;

X_all = X_all_all[i1][i2][:, t];
Y_all = Y_all_all[i1][i2][:, t];
# pairwise distances between cells
incl_self = 0; # include self-pairs of the form [i,i]?
# loop j over range(i+1) to include self-pairs, or range(i) to exclude self-pairs
X_all_paired = np.array([ [X_all[i], X_all[j]] for i in range(len(X_all)) for j in range(i+incl_self)]);
Y_all_paired = np.array([ [Y_all[i], Y_all[j]] for i in range(len(X_all)) for j in range(i+incl_self)]);
def f(x, y):
    return np.sqrt((x[0]-x[1])**2 + (y[0]-y[1])**2)
pairwise_dist_all = np.array(list(map(f, X_all_paired, Y_all_paired )) );

# -- Verify that this yields the pair-wise elements of an array --
# data_all = np.array([[i, j] for i in range(10) for j in range(i+1)]).T;
# plt.scatter( data_all[0], data_all[1]  )
plt.figure(dpi=100)
plt.hist(pairwise_dist_all, bins=15, alpha = 0.4)
plt.show()

In [ ]:
# --- plot for all results ---
t=0;
incl_self = 0; # include self?

pairwise_dist_all_all = [[None]*N_files_per_video for _ in range(N_videos)];
fig, axs = plt.subplots(2, 3, figsize=(12, 5), dpi=100)
for i1 in range(N_videos):
    [ix, iy] = divmod(i1, 2)
    for i2 in range(N_files_per_video):
        X_all = X_all_all[i1][i2][:, t];
        Y_all = Y_all_all[i1][i2][:, t];
        # pairwise distances between cells
        incl_self = 0; # include self-pairs of the form [i,i]?
        # loop j over range(i+1) to include self-pairs, or range(i) to exclude self-pairs
        X_all_paired = np.array([ [X_all[i], X_all[j]] for i in range(len(X_all)) for j in range(i+incl_self)]);
        Y_all_paired = np.array([ [Y_all[i], Y_all[j]] for i in range(len(Y_all)) for j in range(i+incl_self)]);
        def f(x, y):
            return np.sqrt((x[0]-x[1])**2 + (y[0]-y[1])**2)
        pairwise_dist_all = np.array(list(map(f, X_all_paired, Y_all_paired )) );
        
        # store for later use
        pairwise_dist_all_all[i1][i2] = pairwise_dist_all; 
        
        # plot histogram
        axs[ix, iy].hist(pairwise_dist_all, bins=15, alpha = 0.4, density=True)
        
# --- legend ---
for row in range(2):
    for i, this_label in enumerate(names_compare):
        axs[row, 2].plot(np.nan, label=this_label)
    axs[row, 2].legend(loc='upper left')
    axs[row, 2].axis('off')